In [1]:
!pip install tensorflow
!pip install keras

    100% |ââââââââââââââââââââââââââââââââ| 59.0MB 6.3kB/s eta 0:00:01    38% |âââââââââââââ                   | 23.0MB 10.3MB/s eta 0:00:04    40% |âââââââââââââ                   | 23.7MB 7.8MB/s eta 0:00:05    47% |ââââââââââââââââ                | 27.9MB 15.4MB/s eta 0:00:03    55% |ââââââââââââââââââ              | 32.7MB 4.4MB/s eta 0:00:06    82% |âââââââââââââââââââââââââââ     | 48.5MB 10.4MB/s eta 0:00:02    82% |âââââââââââââââââââââââââââ     | 49.0MB 7.0MB/s eta 0:00:02ï¿½ââââââââââââââ | 57.1MB 17.1MB/s eta 0:00:01
    100% |ââââââââââââââââââââââââââââââââ| 92kB 4.0MB/s eta 0:00:01
    100% |â

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [3]:
import os
import gzip
import csv
import numpy as np


In [4]:
import tensorflow as tf

In [5]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

/home/nbuser/anaconda3_23/lib/python3.4/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/home/nbuser/anaconda3_23/lib/python3.4/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [6]:
from six.moves import urllib

In [7]:
print(np.__version__)
print(tf.__version__)

1.14.5
1.10.0


In [8]:
URL_PATH = 'http://ai.stanford.edu/~btaskar/ocr/letter.data.gz'
DOWNLOADED_FILENAME = 'letter.data.gz'

def download_data():
    if not os.path.exists(DOWNLOADED_FILENAME):
        filename, _ = urllib.request.urlretrieve(URL_PATH, DOWNLOADED_FILENAME)
    
    print('Found and verified file from this path: ', URL_PATH)
    print('Downloaded file: ', DOWNLOADED_FILENAME)

In [9]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.callbacks import EarlyStopping

#   Supress warning and informational messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

Using TensorFlow backend.


In [10]:
download_data()

Found and verified file from this path:  http://ai.stanford.edu/~btaskar/ocr/letter.data.gz
Downloaded file:  letter.data.gz


In [11]:
def read_lines():
    with gzip.open(DOWNLOADED_FILENAME, 'rt') as f:
        reader = csv.reader(f, delimiter='\t')
        lines = list(reader)

        return lines

In [12]:
lines = read_lines()

In [13]:
lines[0]

['1',
 'o',
 '2',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '']

In [14]:
len(lines)

52152

In [15]:
def get_features_labels(lines):
    lines = sorted(lines, key=lambda x: int(x[0]))
    data, target = [], []
    
    next_id = -1
    
    word = []
    word_pixels = []

    for line in lines:
         # The index for the next_id column
        next_id = int(line[2])

        # An image for a single character, reshaped
        pixels = np.array([int(x) for x in line[6:134]])
        pixels = pixels.reshape((16, 8))
        
        # Word pixels are a list of 16x8 images which form a single word
        word_pixels.append(pixels)
        
        # Append together the characters which make up a word
        word.append(line[1])
        
        if next_id == -1:
            data.append(word_pixels)
            target.append(word)

            word = []
            word_pixels = []


    return data, target

In [16]:
data, target = get_features_labels(lines)

In [17]:
len(data), len(target)

(6877, 6877)

In [18]:
data[0]

[array([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 1, 1, 0, 0, 0, 0],
        [0, 1, 1, 1, 1, 1, 0, 0],
        [0, 1, 0, 0, 0, 1, 1, 0],
        [1, 1, 0, 0, 0, 0, 1, 1],
        [1, 0, 0, 0, 0, 0, 0, 1],
        [1, 0, 0, 0, 0, 0, 0, 1],
        [1, 0, 0, 0, 0, 0, 0, 1],
        [1, 0, 0, 0, 0, 0, 1, 1],
        [1, 0, 0, 0, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]]), array([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 0, 1, 1, 0, 1, 1],
        [1, 0, 1, 1, 0, 0, 0, 1],
        [1, 0, 1, 1, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0

In [19]:
target[567]

['e', 'v', 'v', 'i', 'n', 'g']

In [20]:
def pad_features_labels(data, target):    
    max_length = max(len(x) for x in target)
    
    # Set up image representations for the empty string (all pixels set to 0)
    padding = np.zeros((16, 8))

    # Pad the image data with the empty string images
    data = [x + ([padding] * (max_length - len(x))) for x in data]
    
    # Pad the words with empty string characters
    target = [x + ([''] * (max_length - len(x))) for x in target]
    
    return np.array(data), np.array(target)

In [21]:
padded_data, padded_target = pad_features_labels(data, target)

In [22]:
padded_data.shape

(6877, 14, 16, 8)

In [23]:
padded_target[:10]

array([['o', 'm', 'm', 'a', 'n', 'd', 'i', 'n', 'g', '', '', '', '', ''],
       ['o', 'm', 'm', 'a', 'n', 'd', 'i', 'n', 'g', '', '', '', '', ''],
       ['o', 'm', 'm', 'a', 'n', 'd', 'i', 'n', 'g', '', '', '', '', ''],
       ['o', 'm', 'm', 'a', 'n', 'd', 'i', 'n', 'g', '', '', '', '', ''],
       ['o', 'm', 'm', 'a', 'n', 'd', 'i', 'n', 'g', '', '', '', '', ''],
       ['o', 'm', 'm', 'a', 'n', 'd', 'i', 'n', 'g', '', '', '', '', ''],
       ['o', 'm', 'm', 'a', 'n', 'd', 'i', 'n', 'g', '', '', '', '', ''],
       ['o', 'm', 'm', 'a', 'n', 'd', 'i', 'n', 'g', '', '', '', '', ''],
       ['o', 'm', 'm', 'a', 'n', 'd', 'i', 'n', 'g', '', '', '', '', ''],
       ['o', 'm', 'm', 'a', 'n', 'd', 'i', 'n', 'g', '', '', '', '', '']],
      dtype='<U1')

In [24]:
padded_target[200:210]

array([['m', 'b', 'r', 'a', 'c', 'e', 's', '', '', '', '', '', '', ''],
       ['m', 'b', 'r', 'a', 'c', 'e', 's', '', '', '', '', '', '', ''],
       ['m', 'b', 'r', 'a', 'c', 'e', 's', '', '', '', '', '', '', ''],
       ['m', 'b', 'r', 'a', 'c', 'e', 's', '', '', '', '', '', '', ''],
       ['m', 'b', 'r', 'a', 'c', 'e', 's', '', '', '', '', '', '', ''],
       ['m', 'b', 'r', 'a', 'c', 'e', 's', '', '', '', '', '', '', ''],
       ['m', 'b', 'r', 'a', 'c', 'e', 's', '', '', '', '', '', '', ''],
       ['m', 'b', 'r', 'a', 'c', 'e', 's', '', '', '', '', '', '', ''],
       ['m', 'b', 'r', 'a', 'c', 'e', 's', '', '', '', '', '', '', ''],
       ['m', 'b', 'r', 'a', 'c', 'e', 's', '', '', '', '', '', '', '']],
      dtype='<U1')

In [25]:
word_length = len(padded_target[0])

In [26]:
word_length

14

In [27]:
padded_data.shape

(6877, 14, 16, 8)

In [28]:
padded_data.shape[:2] + (-1,)

(6877, 14, -1)

In [29]:
reshaped_data = padded_data.reshape(padded_data.shape[:2] + (-1,))

In [30]:
reshaped_data.shape

(6877, 14, 128)

In [31]:
padded_target.shape

(6877, 14)

In [32]:
padded_target.shape + (26,)

(6877, 14, 26)

In [33]:
one_hot_target = np.zeros(padded_target.shape + (26,))

In [34]:
for index, letter in np.ndenumerate(padded_target):
    if letter:
        one_hot_target[index][ord(letter) - ord('a')] = 1

In [35]:
one_hot_target[0]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 

In [36]:
shuffled_indices = np.random.permutation(len(reshaped_data))

shuffled_data = reshaped_data[shuffled_indices]
shuffled_target = one_hot_target[shuffled_indices]

In [37]:
split = int(0.66 * len(shuffled_data))

train_data = shuffled_data[:split]
train_target = shuffled_target[:split]

test_data = shuffled_data[split:]
test_target = shuffled_target[split:]

In [38]:
train_data.shape

(4538, 14, 128)

In [39]:
train_target.shape

(4538, 14, 26)

In [40]:
test_data.shape

(2339, 14, 128)

In [41]:
test_target.shape

(2339, 14, 26)

In [42]:
_, num_steps, num_inputs = train_data.shape

In [43]:
num_inputs

128

In [44]:
np.array(train_target.shape)

array([4538,   14,   26])

In [45]:
num_classes = train_target.shape[2]

In [46]:
num_classes

26

In [47]:
train_target_reshaped=train_target.reshape(train_target.shape[0],-1)
train_target_reshaped.shape

(4538, 364)

In [48]:
test_target_reshaped=test_target.reshape(test_target.shape[0],-1)
test_target_reshaped.shape

(2339, 364)

In [49]:
#run keras model here

from keras.layers import Flatten,LSTMCell,Dropout,GRU
model = Sequential()
#keras.layers.LSTM(units, activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.3, recurrent_dropout=0.3, implementation=1, return_sequences=False, return_state=False, go_backwards=False, stateful=False, unroll=False)
model.add(GRU(units=150,
    input_shape = (14,128),dropout=0.3, recurrent_dropout=0.3,activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None,implementation=1,unroll=True))
model.add(Dropout(0.25))
model.add(Dense(364,activation='sigmoid'))



In [50]:
#keras
model.compile(loss='binary_crossentropy',  
            optimizer='RMSProp',              
            metrics=['accuracy'])


In [51]:
#keras
BATCH_SIZE = 24
EPOCHS = 5
cbk_early_stopping = EarlyStopping(monitor='val_acc', patience=2, mode='max')
model.fit(train_data, train_target_reshaped, BATCH_SIZE, epochs=EPOCHS, 
             validation_data=(test_data, test_target_reshaped),
            callbacks=[cbk_early_stopping] )

Train on 4538 samples, validate on 2339 samples
Epoch 1/5
4538/4538 [==============================] - 20s 4ms/step - loss: 0.1036 - acc: 0.9739 - val_loss: 0.0778 - val_acc: 0.9793
Epoch 2/5
4538/4538 [==============================] - 14s 3ms/step - loss: 0.0743 - acc: 0.9791 - val_loss: 0.0663 - val_acc: 0.9793
Epoch 3/5
4538/4538 [==============================] - 14s 3ms/step - loss: 0.0644 - acc: 0.9791 - val_loss: 0.0568 - val_acc: 0.9796
Epoch 4/5
4538/4538 [==============================] - 15s 3ms/step - loss: 0.0560 - acc: 0.9796 - val_loss: 0.0502 - val_acc: 0.9802
Epoch 5/5
4538/4538 [==============================] - 13s 3ms/step - loss: 0.0497 - acc: 0.9807 - val_loss: 0.0432 - val_acc: 0.9823


In [52]:
#keras
score, acc = model.evaluate(test_data, test_target_reshaped,
                            batch_size=BATCH_SIZE)
print('test score:', score, ' test accuracy:', acc)

2339/2339 [==============================] - 2s 906us/step
test score: 0.043177195990720126  test accuracy: 0.9823419346214105


In [53]:
#tensorflow goes from  here

In [54]:
tf.reset_default_graph()

In [55]:
X = tf.placeholder(tf.float64, [None, num_steps, num_inputs])

y = tf.placeholder(tf.float64, [None, num_steps, num_classes])

In [56]:
# All real characters will have a max value of 1, padded characters will be represented by 0s
used = tf.sign(tf.reduce_max(tf.abs(X), reduction_indices=2))

# Sum up the number of real characters for each word
length = tf.reduce_sum(used, reduction_indices=1)
sequence_length = tf.cast(length, tf.int32)

In [57]:
sequence_length

<tf.Tensor 'Cast:0' shape=(?,) dtype=int32>

In [58]:
num_neurons = 300

In [59]:
cell = tf.nn.rnn_cell.GRUCell(num_neurons)

In [60]:
output, _ = tf.nn.dynamic_rnn(cell, X, dtype=tf.float64, sequence_length=sequence_length)

In [61]:
output.shape

TensorShape([Dimension(None), Dimension(14), Dimension(300)])

In [62]:
weight = tf.Variable(tf.truncated_normal([num_neurons, num_classes], stddev=0.01, dtype=tf.float64))

In [63]:
bias = tf.Variable(tf.constant(0.1, shape=[num_classes], dtype=tf.float64))

In [64]:
flattened_output = tf.reshape(output, [-1, num_neurons])

In [65]:
flattened_output

<tf.Tensor 'Reshape:0' shape=(?, 300) dtype=float64>

In [66]:
logits = tf.matmul(flattened_output, weight) + bias

In [67]:
logits_reshaped = tf.reshape(logits, [-1, num_steps, num_classes])

In [68]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [69]:
loss = tf.reduce_mean(cross_entropy)

In [70]:
mistakes = tf.not_equal(
            tf.argmax(y, 2), tf.argmax(logits_reshaped, 2))
mistakes = tf.cast(mistakes, tf.float64)
mask = tf.sign(tf.reduce_max(tf.abs(y), reduction_indices=2))
mistakes *= mask

In [71]:
mistakes = tf.reduce_sum(mistakes, reduction_indices=1)
mistakes /= tf.cast(sequence_length, tf.float64)

In [72]:
error = tf.reduce_mean(mistakes)

In [73]:
optimizer = tf.train.RMSPropOptimizer(0.002)

In [74]:
gradient = optimizer.compute_gradients(loss)

In [75]:
optimize = optimizer.apply_gradients(gradient)

In [76]:
def batched(data, target, batch_size):
    epoch = 0
    offset = 0
    while True:
        old_offset = offset
        offset = (offset + batch_size) % (target.shape[0] - batch_size)

        # Offset wrapped around to the beginning so new epoch
        if offset < old_offset:
            # New epoch, need to shuffle data
            shuffled_indices = np.random.permutation(len(data))
            
            data = data[shuffled_indices]
            target = target[shuffled_indices]

            epoch += 1

        batch_data = data[offset:(offset + batch_size), :]
        
        batch_target = target[offset:(offset + batch_size), :]

        yield batch_data, batch_target, epoch

In [77]:
batch_size = 20
batches = batched(train_data, train_target, batch_size)

In [78]:
epochs = 5

In [79]:
train_target.shape

(4538, 14, 26)

In [80]:
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())

    for index, batch in enumerate(batches):
        batch_data = batch[0]
        batch_target = batch[1]
    
        epoch = batch[2]

        if epoch >= epochs:
            break
        
        feed = {X: batch_data, y: batch_target}
        train_error, _ = sess.run([error, optimize], feed)
        
        print('{}: {:3.6f}%'.format(index + 1, 100 * train_error))

    test_feed = {X: test_data, y: test_target}
    test_error, _ = sess.run([error, optimize], test_feed)
    
    print('Test error: {:3.6f}%'.format(100 * test_error))

1: 91.039502%
2: 95.327381%
3: 94.518162%
4: 97.267857%
5: 92.013889%
6: 94.285714%
7: 92.058983%
8: 93.303571%
9: 93.273810%
10: 92.706349%
11: 92.460317%
12: 95.311147%
13: 95.183150%
14: 93.535354%
15: 95.041667%
16: 93.136724%
17: 94.173535%
18: 90.448718%
19: 93.823232%
20: 91.047869%
21: 93.286977%
22: 91.647436%
23: 88.918651%
24: 94.384921%
25: 87.625000%
26: 95.041667%
27: 92.308150%
28: 91.123016%
29: 93.318903%
30: 91.917707%
31: 90.496032%
32: 87.388653%
33: 87.304834%
34: 88.094114%
35: 82.867133%
36: 92.140818%
37: 90.368687%
38: 84.541126%
39: 94.363400%
40: 91.505952%
41: 84.569084%
42: 87.162463%
43: 90.519481%
44: 82.241522%
45: 82.644481%
46: 90.932540%
47: 86.301407%
48: 87.905067%
49: 85.585317%
50: 91.678211%
51: 91.545940%
52: 91.356061%
53: 89.486111%
54: 91.934524%
55: 87.490454%
56: 86.856782%
57: 88.200397%
58: 91.691281%
59: 87.630897%
60: 85.015512%
61: 87.077381%
62: 91.602453%
63: 88.821068%
64: 92.073232%
65: 89.683150%
66: 90.611056%
67: 87.329129%
68: 

526: 1.714286%
527: 5.708333%
528: 4.055556%
529: 9.090354%
530: 9.595599%
531: 14.547009%
532: 12.479853%
533: 5.412698%
534: 15.432540%
535: 6.051282%
536: 9.710498%
537: 5.705128%
538: 6.888889%
539: 10.708333%
540: 9.621212%
541: 4.655303%
542: 13.259310%
543: 10.821429%
544: 15.436508%
545: 11.801768%
546: 12.387363%
547: 5.736111%
548: 5.333333%
549: 5.277778%
550: 4.412698%
551: 5.456710%
552: 10.863095%
553: 3.787879%
554: 4.608392%
555: 15.044900%
556: 8.408120%
557: 7.708333%
558: 7.636600%
559: 5.551768%
560: 9.866939%
561: 15.869228%
562: 10.547619%
563: 11.259615%
564: 11.456349%
565: 5.416667%
566: 8.135032%
567: 5.575092%
568: 11.738095%
569: 11.255772%
570: 9.441850%
571: 3.467643%
572: 3.585859%
573: 11.829060%
574: 9.529762%
575: 6.690476%
576: 6.085317%
577: 8.182720%
578: 18.054293%
579: 11.111111%
580: 8.690476%
581: 7.476190%
582: 11.977564%
583: 4.960317%
584: 9.051282%
585: 4.757631%
586: 17.500000%
587: 7.412393%
588: 4.468254%
589: 5.547619%
590: 6.537698%
591

1064: 6.666667%
1065: 5.289072%
1066: 8.468254%
1067: 8.769841%
1068: 2.539683%
1069: 8.144841%
1070: 1.458333%
1071: 7.555556%
1072: 4.111111%
1073: 5.508658%
1074: 2.606838%
1075: 5.103175%
1076: 1.894841%
1077: 4.444444%
1078: 3.541667%
1079: 3.176282%
1080: 6.088980%
1081: 7.162698%
1082: 4.047619%
1083: 3.023504%
1084: 2.815171%
1085: 1.458333%
1086: 5.000000%
1087: 2.845238%
1088: 2.260101%
1089: 1.454545%
1090: 8.625000%
1091: 8.523810%
1092: 11.755952%
1093: 4.472222%
1094: 8.985806%
1095: 10.055556%
1096: 4.333333%
1097: 3.766234%
1098: 3.005952%
1099: 6.736111%
1100: 6.388889%
1101: 8.373016%
1102: 0.555556%
1103: 4.166667%
1104: 2.847222%
1105: 1.625000%
1106: 5.722222%
1107: 4.428571%
1108: 1.666667%
1109: 11.602564%
1110: 8.595238%
1111: 1.654457%
1112: 6.031746%
1113: 1.825397%
1114: 6.500000%
1115: 5.246032%
1116: 5.357143%
1117: 2.222222%
1118: 1.666667%
1119: 5.213675%
1120: 1.555556%
1121: 4.246032%
1122: 5.000000%
1123: 3.413947%
1124: 3.222222%
1125: 3.242063%
1126: